### **SAR Image Visualization:**  
***We are required to make observations about the effect of Windowing on the SAR complex-SLC data.***  

***The code is divided in 3 parts:***  
***1. Loading the file.***  
***2. Applying the window and Fourier transform to obtain a filtered image.***  
***3. Plotting the results for comparison.***

***Code Imports***

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import fft
from sarpy.io.complex.converter import open_complex

***Image data is read using the SICD/NITF file reader from *SarPy* package.***

In [ ]:
filename = "IMG-VV-STRIXB-20220811T004713Z-SMSLC-SICD.nitf"

reader = open_complex(filename)
slc_image_data = reader[:]
n_az, n_rg = slc_image_data.shape
print(f"SLC dtype: {slc_image_data.dtype}, shape: {slc_image_data.shape}")

***This section performs two tasks:***  
***1. It sections the *Hanning* window such that only the center part of the image comes under the window.***  
***2. It obtains the filtered image through series of time to frequency and frequency to time conversions along the range lines.***

In [ ]:
window_frac = 0.5  # part of the window to be used

range_window = np.zeros(n_rg)
center_rbin = (n_rg // 2)   
half_window_width = int((n_rg * window_frac) / 2)  # division by 2 ensures window symmetry
hanning_window_section = np.hanning(2 * half_window_width)
range_window[center_rbin - half_window_width : center_rbin + half_window_width] = hanning_window_section

image_fft = fft.fftshift(fft.fft(slc_image_data, axis=1), axes=1)  # fftshift used to enable multiplication of the low frequency spectra with the window.
windowed_fft = image_fft * range_window[None, :]
filtered_image = fft.ifft(fft.ifftshift(windowed_fft, axes=1), n_rg, axis=1)  # ifftshift performed before ifft to shift the spectrum back to its original place.


***This section contains Display function which plots the *power(dB)* of the input images for comparison.  
And the dynamic range (Vmin, Vmax values) of the images while plotting have been set to default *(-40,20)* to account for weak (vegetation, shadows, water body etc.) and strong scatterers (metallic surfaces, buildings, ships) both.*** 

In [ ]:
def plot_images(original_image, filtered_image, row_range=None, col_range=None):
    """
    Display full or zoomed images (original vs filtered).
    """
    to_zoom = bool(row_range and col_range)

    if to_zoom:
        extent = [col_range[1], col_range[0], row_range[0], row_range[1]]
    else:
        extent = [0, n_rg, n_az, 0]  # full range

    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    plt.imshow(20*np.log10(original_image + 1e-12), cmap = "gray", vmin = -40, vmax = 20, extent = extent)
    plt.title("Original" + (" (Zoomed)" if (to_zoom) else ""))
    plt.xlabel("Azimuth"); plt.ylabel("Range");

    plt.subplot(1, 2, 2)
    plt.imshow(20*np.log10(filtered_image + 1e-12), cmap = "gray", vmin = -40, vmax = 20, extent = extent)
    plt.title("Filtered" + (" (Zoomed)" if (to_zoom) else ""))
    plt.xlabel("Azimuth"); plt.ylabel("Range")

    plt.tight_layout()
    plt.show()

***This section passes the magnitude of the full and zoomed-in original and filtered images to the display function.*** 

In [ ]:
original_image_magnitude = np.abs(slc_image_data)  # original magnitude
filtered_image_magnitude = np.abs(filtered_image)  # filtered magnitude

plot_images(original_image_magnitude, filtered_image_magnitude)

# range and azimuth indices to gain a close-up look of Busan harbor. 
col_range = (13150, 14450)
row_range = (925, 2025)

zoomed_original_magnitude = original_image_magnitude[row_range[0]:row_range[1], col_range[0]:col_range[1]]
zoomed_filtered_magnitude = filtered_image_magnitude[row_range[0]:row_range[1], col_range[0]:col_range[1]]

plot_images(zoomed_original_magnitude, zoomed_filtered_magnitude, row_range=row_range, col_range=col_range)
